# **Install face_net**

In [2]:
!pip install facenet-pytorch

# Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing libraries

In [4]:
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch.nn.functional as F

# gpu/cpu

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


# loading pretrained facenet

In [6]:
mtcnn = MTCNN(image_size=160, margin=0, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

# utilizing the pretraied model process

In [7]:
def extract_face(image_path):
  #opens the images and detect faces by using mtcnn
    img = Image.open(image_path)
    face_tensor = mtcnn(img)
    if face_tensor is None:
       raise ValueError(f"No image detected in {image_path}")
    return face_tensor

In [8]:
def get_embedding(face_tensor):
  #extracts feature vector from each image
    face_tensor = face_tensor.unsqueeze(0).to(device)
    embedding = resnet(face_tensor)
    return embedding.detach().cpu()[0]

In [ ]:
reference_images = [
    '/content/drive/MyDrive/walterbishop/images.jfif',
    '/content/drive/MyDrive/walterbishop/13663-1532336916.jpg',
    '/content/drive/MyDrive/walterbishop/HkjAj.jpg',
    '/content/drive/MyDrive/walterbishop/YyZMr.jpg',
    '/content/drive/MyDrive/walterbishop/fringe-john-noble-season-4.jpg',
    '/content/drive/MyDrive/walterbishop/John-Noble-Walter-Bishop.jpg',
]

test_image_path = '/content/drive/MyDrive/walterbishop/HkjAj.jpg'
ref_embeddings = []
for img_path in reference_images:
    try:
       face = extract_face(img_path)
       emb = get_embedding(face)
       ref_embeddings.append(emb)
    except Exception as e:
       print(f"Error processing {img_path}: {e}")
if not ref_embeddings:
    raise ValueError("No valid reference embeddings were extracted. Please check your images.")
reference_embedding = torch.stack(ref_embeddings).mean(0)

In [ ]:
try:
    test_face = extract_face(test_image_path)
    test_embedding = get_embedding(test_face)
    print("Test image processed successfully.")
except Exception as e:
    raise ValueError(f"Error processing the test image: {e}")

Test image processed successfully.


In [ ]:
euclidean_distance = torch.norm(reference_embedding - test_embedding)
cosine_similarity = F.cosine_similarity(reference_embedding.unsqueeze(0),
                                        test_embedding.unsqueeze(0))[0]

print("\n----- Verification Results -----")
print(f"Euclidean distance: {euclidean_distance.item():.4f}")
print(f"Cosine similarity: {cosine_similarity.item():.4f}")


----- Verification Results -----
Euclidean distance: 0.5487
Cosine similarity: 0.8370


In [ ]:
euclidean_threshold = 1.0
cosine_threshold = 0.8

In [ ]:
if euclidean_distance < euclidean_threshold:
    print("Test image likely belongs to Walter Bishop (based on Euclidean distance).")
else:
    print("Test image does not appear to belong to Walter Bishop (based on Euclidean distance).")

Test image likely belongs to Walter Bishop (based on Euclidean distance).


In [ ]:
if cosine_similarity > cosine_threshold:
    print("Test image likely belongs to Walter Bishop (based on Cosine similarity).")
else:
    print("Test image does not appear to belong to Walter Bishop (based on Cosine similarity).")

Test image likely belongs to Walter Bishop (based on Cosine similarity).


# other image

In [10]:
reference_images = [
    '/content/drive/MyDrive/walterbishop/images.jfif',
    '/content/drive/MyDrive/walterbishop/13663-1532336916.jpg',
    '/content/drive/MyDrive/walterbishop/HkjAj.jpg',
    '/content/drive/MyDrive/walterbishop/YyZMr.jpg',
    '/content/drive/MyDrive/walterbishop/fringe-john-noble-season-4.jpg',
    '/content/drive/MyDrive/walterbishop/John-Noble-Walter-Bishop.jpg',
]
test_image_path = '/content/drive/MyDrive/oliviadunham/olivia-dunham-16.jpg'
ref_embeddings = []
for img_path in reference_images:
    try:
       face = extract_face(img_path)
       emb = get_embedding(face)
       ref_embeddings.append(emb)
    except Exception as e:
       print(f"Error processing {img_path}: {e}")
if not ref_embeddings:
    raise ValueError("No valid reference embeddings were extracted. Please check your images.")
reference_embedding = torch.stack(ref_embeddings).mean(0)

In [11]:
try:
    test_face = extract_face(test_image_path)
    test_embedding = get_embedding(test_face)
    print("Test image processed successfully.")
except Exception as e:
    raise ValueError(f"Error processing the test image: {e}")

Test image processed successfully.


In [12]:
euclidean_distance = torch.norm(reference_embedding - test_embedding)
cosine_similarity = F.cosine_similarity(reference_embedding.unsqueeze(0),
                                        test_embedding.unsqueeze(0))[0]

print("\n----- Verification Results -----")
print(f"Euclidean distance: {euclidean_distance.item():.4f}")
print(f"Cosine similarity: {cosine_similarity.item():.4f}")


----- Verification Results -----
Euclidean distance: 1.4007
Cosine similarity: -0.1090


In [13]:
euclidean_threshold = 1.0
cosine_threshold = 0.8

In [14]:
if euclidean_distance < euclidean_threshold:
    print("Test image likely belongs to Walter Bishop (based on Euclidean distance).")
else:
    print("Test image does not appear to belong to Walter Bishop (based on Euclidean distance).")

Test image does not appear to belong to Walter Bishop (based on Euclidean distance).


In [15]:
if cosine_similarity > cosine_threshold:
    print("Test image likely belongs to Walter Bishop (based on Cosine similarity).")
else:
    print("Test image does not appear to belong to Walter Bishop (based on Cosine similarity).")

Test image does not appear to belong to Walter Bishop (based on Cosine similarity).
